In [1]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
df = pd.read_excel ('Data for Capstone.xlsx')
df.head()

,City,Zip Code,Longitude,Latitude
0,Zürich,8001,8.530807,47.382824
1,Zürich,8002,8.539550,47.377060
2,Zürich,8003,8.539550,47.377060
3,Zürich,8004,8.530807,47.382824
4,Zürich,8005,8.530807,47.382824


In [3]:
CLIENT_ID = 'RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK' # your Foursquare ID
CLIENT_SECRET = 'D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK
CLIENT_SECRET:D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB


In [4]:
search_query = 'Food'
radius = 1000
print(search_query + ' .... OK!')

Food .... OK!


In [5]:
# KREIS1
latitude = 47.370278
longitude = 8.541667
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK&client_secret=D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB&ll=47.370278,8.541667&v=20180605&query=Food&radius=1000&limit=100'

In [6]:
results = requests.get(url).json()

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,560595f7498eb0f06171956e,Food from Farmers,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",v-1591131861,False,Langstrasse 211,Josefstrasse,47.368502,8.545671,"[{'label': 'display', 'lat': 47.36850193220842...",360,8005,CH,Zürich,Zürich,Schweiz,"[Langstrasse 211 (Josefstrasse), 8005 Zürich, ...",NaN,NaN
1,5357cc16498e0e40db63f54a,Jelmoli Food Market,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",v-1591131861,False,NaN,NaN,47.374083,8.537229,"[{'label': 'display', 'lat': 47.3740827107709,...",539,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN,NaN
2,4e87185182316be403a7f640,Hong-Kong Food Paradise,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1591131861,False,Seidengasse 1,NaN,47.374042,8.536562,"[{'label': 'display', 'lat': 47.37404169003569...",568,8001,CH,Zürich,Zürich,Schweiz,"[Seidengasse 1, 8001 Zürich, Schweiz]",NaN,NaN
3,5bac7c428b98fd0039448a15,Food Station,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1591131861,False,Bahnhofplatz 15,NaN,47.378204,8.535982,"[{'label': 'display', 'lat': 47.378204, 'lng':...",980,NaN,CH,NaN,NaN,Schweiz,"[Bahnhofplatz 15, Schweiz]",NaN,NaN
4,5cc1c6a4cabcff002ce22ad4,Coop Supermarkt City Zürich St. Annahof Food,[],v-1591131861,False,Bahnhofstrasse 57,NaN,47.372902,8.538024,"[{'label': 'display', 'lat': 47.37290177, 'lng...",400,8001,CH,Zürich,Zürich,Schweiz,"[Bahnhofstrasse 57, 8001 Zürich, Schweiz]",NaN,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Food from Farmers,Burger Joint,Langstrasse 211,Josefstrasse,47.368502,8.545671,"[{'label': 'display', 'lat': 47.36850193220842...",360,8005,CH,Zürich,Zürich,Schweiz,"[Langstrasse 211 (Josefstrasse), 8005 Zürich, ...",NaN,560595f7498eb0f06171956e
1,Jelmoli Food Market,Market,NaN,NaN,47.374083,8.537229,"[{'label': 'display', 'lat': 47.3740827107709,...",539,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN,5357cc16498e0e40db63f54a
2,Hong-Kong Food Paradise,Chinese Restaurant,Seidengasse 1,NaN,47.374042,8.536562,"[{'label': 'display', 'lat': 47.37404169003569...",568,8001,CH,Zürich,Zürich,Schweiz,"[Seidengasse 1, 8001 Zürich, Schweiz]",NaN,4e87185182316be403a7f640
3,Food Station,Food Court,Bahnhofplatz 15,NaN,47.378204,8.535982,"[{'label': 'display', 'lat': 47.378204, 'lng':...",980,NaN,CH,NaN,NaN,Schweiz,"[Bahnhofplatz 15, Schweiz]",NaN,5bac7c428b98fd0039448a15
4,Coop Supermarkt City Zürich St. Annahof Food,None,Bahnhofstrasse 57,NaN,47.372902,8.538024,"[{'label': 'display', 'lat': 47.37290177, 'lng...",400,8001,CH,Zürich,Zürich,Schweiz,"[Bahnhofstrasse 57, 8001 Zürich, Schweiz]",NaN,5cc1c6a4cabcff002ce22ad4
5,Coop Supermarkt City Zürich Bellevue Food,None,Theaterstrasse 18,NaN,47.367308,8.546079,"[{'label': 'display', 'lat': 47.367308, 'lng':...",468,8024,CH,Zürich,Zürich,Schweiz,"[Theaterstrasse 18, 8024 Zürich, Schweiz]",NaN,5cc1c8e44a1cc0002c7d8132
6,Viet Street Food,Vietnamese Restaurant,Usteristrasse 21,NaN,47.376122,8.536371,"[{'label': 'display', 'lat': 47.376122, 'lng':...",763,NaN,CH,Zürich,Zürich,Schweiz,"[Usteristrasse 21, Zürich, Schweiz]",NaN,5e0b4830456d3b000815955b
7,Vee's Bistro - Thai Food - Take away,Thai Restaurant,Alfred Escher-Strasse 11,NaN,47.364968,8.532719,"[{'label': 'display', 'lat': 47.36496831906324...",896,8002,CH,Zürich,Zürich,Schweiz,"[Alfred Escher-Strasse 11, 8002 Zürich, Schweiz]",NaN,5595fd46498e7e817b7f5605
8,food&lab,Cafeteria,Rämistr. 101,NaN,47.378327,8.548848,"[{'label': 'display', 'lat': 47.37832683213607...",1046,NaN,CH,Zürich,Zürich,Schweiz,"[Rämistr. 101, Zürich, Schweiz]",Oberstrass,4b5841fff964a520e24f28e3
9,Thai Food Corner GmbH,None,NaN,NaN,47.379329,8.535520,"[{'label': 'display', 'lat': 47.37932922752695...",1109,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN,4f8d5903e4b0e5ed77fe6a91


In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [56]:
# KREIS2
search_query = 'restaurant'
radius = 1500
print(search_query + ' .... OK!')
latitude = 47.3375
longitude = 8.520833
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK&client_secret=D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB&ll=47.3375,8.520833&v=20180605&query=restaurant&radius=1500&limit=100'

In [57]:
results1 = requests.get(url).json()

In [58]:
# assign relevant part of JSON to venues
venues1 = results1['response']['venues']

# tranform venues into a dataframe
dataframe1 = pd.json_normalize(venues1)
dataframe1

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,54d10fd7498e22dcd5554932,Restaurant Mellow,[],v-1591132790,False,Albisstrasse 84,47.340591,8.530216,"[{'label': 'display', 'lat': 47.340591, 'lng':...",787,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 84, 8038 Zürich, Schweiz]",147070151
1,4da575440437667c9b2b984e,Restaurant Medina,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,Albisstrasse 72,47.342189,8.530554,"[{'label': 'display', 'lat': 47.34218895149141...",900,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 72, 8038 Zürich, Schweiz]",NaN
2,4bbf01dd98f495212df6d163,SV Restaurant Manegg,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,NaN,47.338563,8.518558,"[{'label': 'display', 'lat': 47.33856296516321...",208,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN
3,4b05888af964a520afcc22e3,Restaurant Bürgli,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,Kilchbergstr. 15,47.343048,8.533720,"[{'label': 'display', 'lat': 47.34304818849731...",1151,8038,CH,Zürich,Zürich,Schweiz,"[Kilchbergstr. 15, 8038 Zürich, Schweiz]",NaN
4,4b83b36cf964a520730d31e3,Restaurant Casa-Nostrana,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1591132790,False,Albisstrasse 25,47.344131,8.531738,"[{'label': 'display', 'lat': 47.34413104786304...",1105,NaN,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 25, Zürich, Schweiz]",NaN
5,4e6f2dc9b0fbacf2ada29857,Restaurant Renggergut,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,NaN,47.344093,8.532964,"[{'label': 'display', 'lat': 47.34409332275390...",1173,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN
6,4b5ebc95f964a520319729e3,Restaurant zur rote Buech,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,Seestrasse 361,47.345600,8.533525,"[{'label': 'display', 'lat': 47.34560000993326...",1315,NaN,CH,Zürich,Zürich,Schweiz,"[Seestrasse 361, Zürich, Schweiz]",NaN
7,5af09361bed483002c28c903,Restaurant Tennisclub Seeblick,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591132790,False,NaN,47.333117,8.538267,"[{'label': 'display', 'lat': 47.333117, 'lng':...",1402,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN
8,577d382a498e7549bd9cc42f,WINIT's Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1591132790,False,Albisstrasse 29,47.344076,8.531424,"[{'label': 'display', 'lat': 47.344076, 'lng':...",1083,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 29, 8038 Zürich, Schweiz]",NaN
9,5a48d7272be4257246b49ee8,Yan-Ruyi Chinesisches Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1591132790,False,NaN,47.344070,8.532312,"[{'label': 'display', 'lat': 47.34407, 'lng': ...",1133,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN


In [59]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe1_filtered = dataframe1.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe1_filtered['categories'] = dataframe1_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe1_filtered.columns = [column.split('.')[-1] for column in dataframe1_filtered.columns]

dataframe1_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Restaurant Mellow,None,Albisstrasse 84,47.340591,8.530216,"[{'label': 'display', 'lat': 47.340591, 'lng':...",787,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 84, 8038 Zürich, Schweiz]",54d10fd7498e22dcd5554932
1,Restaurant Medina,Restaurant,Albisstrasse 72,47.342189,8.530554,"[{'label': 'display', 'lat': 47.34218895149141...",900,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 72, 8038 Zürich, Schweiz]",4da575440437667c9b2b984e
2,SV Restaurant Manegg,Restaurant,NaN,47.338563,8.518558,"[{'label': 'display', 'lat': 47.33856296516321...",208,NaN,CH,NaN,NaN,Schweiz,[Schweiz],4bbf01dd98f495212df6d163
3,Restaurant Bürgli,Restaurant,Kilchbergstr. 15,47.343048,8.533720,"[{'label': 'display', 'lat': 47.34304818849731...",1151,8038,CH,Zürich,Zürich,Schweiz,"[Kilchbergstr. 15, 8038 Zürich, Schweiz]",4b05888af964a520afcc22e3
4,Restaurant Casa-Nostrana,Italian Restaurant,Albisstrasse 25,47.344131,8.531738,"[{'label': 'display', 'lat': 47.34413104786304...",1105,NaN,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 25, Zürich, Schweiz]",4b83b36cf964a520730d31e3
5,Restaurant Renggergut,Restaurant,NaN,47.344093,8.532964,"[{'label': 'display', 'lat': 47.34409332275390...",1173,NaN,CH,NaN,NaN,Schweiz,[Schweiz],4e6f2dc9b0fbacf2ada29857
6,Restaurant zur rote Buech,Restaurant,Seestrasse 361,47.345600,8.533525,"[{'label': 'display', 'lat': 47.34560000993326...",1315,NaN,CH,Zürich,Zürich,Schweiz,"[Seestrasse 361, Zürich, Schweiz]",4b5ebc95f964a520319729e3
7,Restaurant Tennisclub Seeblick,Restaurant,NaN,47.333117,8.538267,"[{'label': 'display', 'lat': 47.333117, 'lng':...",1402,NaN,CH,NaN,NaN,Schweiz,[Schweiz],5af09361bed483002c28c903
8,WINIT's Restaurant,Thai Restaurant,Albisstrasse 29,47.344076,8.531424,"[{'label': 'display', 'lat': 47.344076, 'lng':...",1083,8038,CH,Zürich,Zürich,Schweiz,"[Albisstrasse 29, 8038 Zürich, Schweiz]",577d382a498e7549bd9cc42f
9,Yan-Ruyi Chinesisches Restaurant,Chinese Restaurant,NaN,47.344070,8.532312,"[{'label': 'display', 'lat': 47.34407, 'lng': ...",1133,NaN,CH,NaN,NaN,Schweiz,[Schweiz],5a48d7272be4257246b49ee8


In [60]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kreis2',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map1)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe1_filtered.lat, dataframe1_filtered.lng, dataframe1_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map1)

# display map
venues_map1

In [71]:
# KREIS3
search_query = 'restaurant'
radius = 1500
print(search_query + ' .... OK!')
latitude = 47.361944
longitude = 8.514722
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK&client_secret=D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB&ll=47.361944,8.514722&v=20180605&query=restaurant&radius=1500&limit=100'

In [72]:
results2 = requests.get(url).json()

In [73]:
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = pd.json_normalize(venues2)
dataframe2

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,5165a91ee4b0c547d16fca83,Restaurant Hopfenau,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133145,False,Hopfenstrasse 19,47.365205,8.519986,"[{'label': 'display', 'lat': 47.36520492776915...",537,8045,CH,Zürich,Zürich,Schweiz,"[Hopfenstrasse 19, 8045 Zürich, Schweiz]",NaN,NaN,NaN
1,4cf3678e7bf3b60cb9d36a7f,Restaurant Eat&Meet,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1591133145,False,Binzring 17,47.361802,8.510070,"[{'label': 'display', 'lat': 47.36180195165452...",351,8045,CH,Zürich,Zürich,Schweiz,"[Binzring 17, 8045 Zürich, Schweiz]",NaN,NaN,NaN
2,4dcce3bacc3f07eb1efee990,Restaurant Orangerie,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",v-1591133145,False,Engimattstr. 14,47.359480,8.524874,"[{'label': 'display', 'lat': 47.35948010766192...",813,8002,CH,Zürich,Zürich,Schweiz,"[Engimattstr. 14 (Hotel Engimatt), 8002 Zürich...",Hotel Engimatt,NaN,NaN
3,4b439110f964a5205ce325e3,Restaurant Bederhof,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133145,False,Brandschenkestr. 177,47.362397,8.526789,"[{'label': 'display', 'lat': 47.36239667110947...",911,8002,CH,Zürich,Zürich,Schweiz,"[Brandschenkestr. 177 (Bederstr.), 8002 Zürich...",Bederstr.,NaN,NaN
4,4dd64b51d22d38ef42cb0d04,Migros Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133145,False,Giesshübelstr. 65,47.358242,8.517387,"[{'label': 'display', 'lat': 47.35824231554161...",458,8045,CH,Zürich,Zürich,Schweiz,"[Giesshübelstr. 65, 8045 Zürich, Schweiz]",NaN,NaN,NaN
5,4cf5f10c1801a14319b7e6d4,Rampe Süd Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1591133145,False,Four Points Hotel,47.359240,8.522965,"[{'label': 'display', 'lat': 47.35923995533381...",690,8045,CH,Zürich,Zürich,Schweiz,"[Four Points Hotel (Kalandergasse 1), 8045 Zür...",Kalandergasse 1,NaN,NaN
6,4c878219e602b1f7f02ec87a,Coop Restaurant Zürich Wiedikon,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",v-1591133145,False,Birmensdorferstrasse 200,47.370746,8.515905,"[{'label': 'display', 'lat': 47.37074636, 'lng...",983,8003,CH,Zürich,Zürich,Schweiz,"[Birmensdorferstrasse 200, 8003 Zürich, Schweiz]",NaN,537196507,NaN
7,4d958142c19fb60c7a0a6865,Migros Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133145,False,Birmensdorferstr. 320,47.369931,8.508418,"[{'label': 'display', 'lat': 47.36993134937684...",1008,8055,CH,Zürich,Zürich,Schweiz,"[Birmensdorferstr. 320, 8055 Zürich, Schweiz]",NaN,NaN,NaN
8,5b7042d6dff8150039cc3938,Restaurant Bar Cafe Eyhof,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133145,False,NaN,47.368526,8.495234,"[{'label': 'display', 'lat': 47.368526, 'lng':...",1641,8047,CH,Zürich,Zürich,Schweiz,"[8047 Zürich, Schweiz]",NaN,NaN,Albisrieden
9,5236dcb011d2fa11b2b7a442,Restaurant Nachtjäger,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133145,False,Badenerstr. 310,47.377378,8.512408,"[{'label': 'display', 'lat': 47.37737792805111...",1726,NaN,CH,Zürich,Zürich,Schweiz,"[Badenerstr. 310, Zürich, Schweiz]",NaN,NaN,Sihlfeld


In [74]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe2_filtered = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe2_filtered['categories'] = dataframe2_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe2_filtered.columns = [column.split('.')[-1] for column in dataframe2_filtered.columns]

dataframe2_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Restaurant Hopfenau,Restaurant,Hopfenstrasse 19,47.365205,8.519986,"[{'label': 'display', 'lat': 47.36520492776915...",537,8045,CH,Zürich,Zürich,Schweiz,"[Hopfenstrasse 19, 8045 Zürich, Schweiz]",NaN,NaN,5165a91ee4b0c547d16fca83
1,Restaurant Eat&Meet,Cafeteria,Binzring 17,47.361802,8.510070,"[{'label': 'display', 'lat': 47.36180195165452...",351,8045,CH,Zürich,Zürich,Schweiz,"[Binzring 17, 8045 Zürich, Schweiz]",NaN,NaN,4cf3678e7bf3b60cb9d36a7f
2,Restaurant Orangerie,French Restaurant,Engimattstr. 14,47.359480,8.524874,"[{'label': 'display', 'lat': 47.35948010766192...",813,8002,CH,Zürich,Zürich,Schweiz,"[Engimattstr. 14 (Hotel Engimatt), 8002 Zürich...",Hotel Engimatt,NaN,4dcce3bacc3f07eb1efee990
3,Restaurant Bederhof,Swiss Restaurant,Brandschenkestr. 177,47.362397,8.526789,"[{'label': 'display', 'lat': 47.36239667110947...",911,8002,CH,Zürich,Zürich,Schweiz,"[Brandschenkestr. 177 (Bederstr.), 8002 Zürich...",Bederstr.,NaN,4b439110f964a5205ce325e3
4,Migros Restaurant,Restaurant,Giesshübelstr. 65,47.358242,8.517387,"[{'label': 'display', 'lat': 47.35824231554161...",458,8045,CH,Zürich,Zürich,Schweiz,"[Giesshübelstr. 65, 8045 Zürich, Schweiz]",NaN,NaN,4dd64b51d22d38ef42cb0d04
5,Rampe Süd Restaurant,Breakfast Spot,Four Points Hotel,47.359240,8.522965,"[{'label': 'display', 'lat': 47.35923995533381...",690,8045,CH,Zürich,Zürich,Schweiz,"[Four Points Hotel (Kalandergasse 1), 8045 Zür...",Kalandergasse 1,NaN,4cf5f10c1801a14319b7e6d4
6,Coop Restaurant Zürich Wiedikon,Supermarket,Birmensdorferstrasse 200,47.370746,8.515905,"[{'label': 'display', 'lat': 47.37074636, 'lng...",983,8003,CH,Zürich,Zürich,Schweiz,"[Birmensdorferstrasse 200, 8003 Zürich, Schweiz]",NaN,NaN,4c878219e602b1f7f02ec87a
7,Migros Restaurant,Restaurant,Birmensdorferstr. 320,47.369931,8.508418,"[{'label': 'display', 'lat': 47.36993134937684...",1008,8055,CH,Zürich,Zürich,Schweiz,"[Birmensdorferstr. 320, 8055 Zürich, Schweiz]",NaN,NaN,4d958142c19fb60c7a0a6865
8,Restaurant Bar Cafe Eyhof,Restaurant,NaN,47.368526,8.495234,"[{'label': 'display', 'lat': 47.368526, 'lng':...",1641,8047,CH,Zürich,Zürich,Schweiz,"[8047 Zürich, Schweiz]",NaN,Albisrieden,5b7042d6dff8150039cc3938
9,Restaurant Nachtjäger,Swiss Restaurant,Badenerstr. 310,47.377378,8.512408,"[{'label': 'display', 'lat': 47.37737792805111...",1726,NaN,CH,Zürich,Zürich,Schweiz,"[Badenerstr. 310, Zürich, Schweiz]",NaN,Sihlfeld,5236dcb011d2fa11b2b7a442


In [76]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kreis3',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)

# display map
venues_map2

In [77]:
# KREIS4
search_query = 'restaurant'
radius = 1500
print(search_query + ' .... OK!')
latitude = 47.3775
longitude = 8.521111
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=RLM4O1CAFIWXLU0OLBV2Z33DT45BIULLHUNEYKM4LMR02VUK&client_secret=D5G4H1H3PSWZWTTET3IAO5P55H4CJ2X4XWTIKLTBU11QBHQB&ll=47.3775,8.521111&v=20180605&query=restaurant&radius=1500&limit=100'

In [78]:
results3 = requests.get(url).json()

In [79]:
# assign relevant part of JSON to venues
venues3 = results3['response']['venues']

# tranform venues into a dataframe
dataframe3 = pd.json_normalize(venues3)
dataframe3

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,location.crossStreet
0,4b7418d2f964a520bcc72de3,Restaurant B,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133462,False,Hohlstrasse 67,47.377800,8.524277,"[{'label': 'display', 'lat': 47.37779982289422...",240,8004,CH,Zürich,Zürich,Schweiz,"[Hohlstrasse 67, 8004 Zürich, Schweiz]",NaN,NaN,NaN
1,56fec063498e86f28f9934ef,Schönau Bar Restaurant,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",v-1591133462,False,Hohlstrasse 78,47.378234,8.524223,"[{'label': 'display', 'lat': 47.37823394884292...",248,8004,CH,Zürich,Zürich,Schweiz,"[Hohlstrasse 78, 8004 Zürich, Schweiz]",177575466,NaN,NaN
2,4bb0ef7df964a520cb6b3ce3,Restaurant Reithalle,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133462,False,Gessner-Allee 8,47.374416,8.534619,"[{'label': 'display', 'lat': 47.37441567682283...",1074,8001,CH,Zürich,Zürich,Schweiz,"[Gessner-Allee 8, 8001 Zürich, Schweiz]",NaN,NaN,NaN
3,4d94d8bcca3f6ea80f3cd334,Hardhof Café & Restaurant,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133462,False,Badenerstrasse 344,47.378126,8.510778,"[{'label': 'display', 'lat': 47.37812551106540...",782,8004,CH,Zürich,Zürich,Schweiz,"[Badenerstrasse 344, 8004 Zürich, Schweiz]",NaN,NaN,NaN
4,5236dcb011d2fa11b2b7a442,Restaurant Nachtjäger,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",v-1591133462,False,Badenerstr. 310,47.377378,8.512408,"[{'label': 'display', 'lat': 47.37737792805111...",656,NaN,CH,Zürich,Zürich,Schweiz,"[Badenerstr. 310, Zürich, Schweiz]",NaN,Sihlfeld,NaN
5,4c88b3c9bbec6dcb0fbdd558,Restaurant St. Meinrad,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133462,False,NaN,47.377554,8.521318,"[{'label': 'display', 'lat': 47.37755432530285...",16,NaN,CH,Zürich,Zürich,Schweiz,"[Zürich, Schweiz]",NaN,NaN,NaN
6,4d270450849f37042f726441,Restaurant Stauffachertor,"[{'id': '52e81612bcbc57f1066b7a09', 'name': 'F...",v-1591133462,False,Werdstr. 6,47.372680,8.530370,"[{'label': 'display', 'lat': 47.37267969448555...",880,8004,CH,Zürich,Zürich,Schweiz,"[Werdstr. 6, 8004 Zürich, Schweiz]",NaN,NaN,NaN
7,4eb2f06130f85a3c6d89de74,Restaurant Valentin's,"[{'id': '4f04af1f2fb6e1c99f3db0bb', 'name': 'T...",v-1591133462,False,Gasometerstrasse 36,47.384740,8.529904,"[{'label': 'display', 'lat': 47.384740237094, ...",1043,8005,CH,Zürich,Zürich,Schweiz,"[Gasometerstrasse 36, 8005 Zürich, Schweiz]",NaN,NaN,NaN
8,5165a91ee4b0c547d16fca83,Restaurant Hopfenau,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591133462,False,Hopfenstrasse 19,47.365205,8.519986,"[{'label': 'display', 'lat': 47.36520492776915...",1371,8045,CH,Zürich,Zürich,Schweiz,"[Hopfenstrasse 19, 8045 Zürich, Schweiz]",NaN,NaN,NaN
9,4f3b6e7fe4b022039bef6654,Restaurant Conrad,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1591133462,False,Sihlstrasse 31,47.372904,8.536534,"[{'label': 'display', 'lat': 47.372904, 'lng':...",1270,8001,CH,Zürich,Zürich,Schweiz,"[Sihlstrasse 31 (Hotel Glockenhof), 8001 Züric...",NaN,City,Hotel Glockenhof


In [80]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe3.columns if col.startswith('location.')] + ['id']
dataframe3_filtered = dataframe3.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe3_filtered['categories'] = dataframe3_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe3_filtered.columns = [column.split('.')[-1] for column in dataframe3_filtered.columns]

dataframe3_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Restaurant B,Swiss Restaurant,Hohlstrasse 67,47.377800,8.524277,"[{'label': 'display', 'lat': 47.37779982289422...",240,8004,CH,Zürich,Zürich,Schweiz,"[Hohlstrasse 67, 8004 Zürich, Schweiz]",NaN,NaN,4b7418d2f964a520bcc72de3
1,Schönau Bar Restaurant,Mediterranean Restaurant,Hohlstrasse 78,47.378234,8.524223,"[{'label': 'display', 'lat': 47.37823394884292...",248,8004,CH,Zürich,Zürich,Schweiz,"[Hohlstrasse 78, 8004 Zürich, Schweiz]",NaN,NaN,56fec063498e86f28f9934ef
2,Restaurant Reithalle,Swiss Restaurant,Gessner-Allee 8,47.374416,8.534619,"[{'label': 'display', 'lat': 47.37441567682283...",1074,8001,CH,Zürich,Zürich,Schweiz,"[Gessner-Allee 8, 8001 Zürich, Schweiz]",NaN,NaN,4bb0ef7df964a520cb6b3ce3
3,Hardhof Café & Restaurant,Swiss Restaurant,Badenerstrasse 344,47.378126,8.510778,"[{'label': 'display', 'lat': 47.37812551106540...",782,8004,CH,Zürich,Zürich,Schweiz,"[Badenerstrasse 344, 8004 Zürich, Schweiz]",NaN,NaN,4d94d8bcca3f6ea80f3cd334
4,Restaurant Nachtjäger,Swiss Restaurant,Badenerstr. 310,47.377378,8.512408,"[{'label': 'display', 'lat': 47.37737792805111...",656,NaN,CH,Zürich,Zürich,Schweiz,"[Badenerstr. 310, Zürich, Schweiz]",Sihlfeld,NaN,5236dcb011d2fa11b2b7a442
5,Restaurant St. Meinrad,Restaurant,NaN,47.377554,8.521318,"[{'label': 'display', 'lat': 47.37755432530285...",16,NaN,CH,Zürich,Zürich,Schweiz,"[Zürich, Schweiz]",NaN,NaN,4c88b3c9bbec6dcb0fbdd558
6,Restaurant Stauffachertor,Fondue Restaurant,Werdstr. 6,47.372680,8.530370,"[{'label': 'display', 'lat': 47.37267969448555...",880,8004,CH,Zürich,Zürich,Schweiz,"[Werdstr. 6, 8004 Zürich, Schweiz]",NaN,NaN,4d270450849f37042f726441
7,Restaurant Valentin's,Turkish Restaurant,Gasometerstrasse 36,47.384740,8.529904,"[{'label': 'display', 'lat': 47.384740237094, ...",1043,8005,CH,Zürich,Zürich,Schweiz,"[Gasometerstrasse 36, 8005 Zürich, Schweiz]",NaN,NaN,4eb2f06130f85a3c6d89de74
8,Restaurant Hopfenau,Restaurant,Hopfenstrasse 19,47.365205,8.519986,"[{'label': 'display', 'lat': 47.36520492776915...",1371,8045,CH,Zürich,Zürich,Schweiz,"[Hopfenstrasse 19, 8045 Zürich, Schweiz]",NaN,NaN,5165a91ee4b0c547d16fca83
9,Restaurant Conrad,Diner,Sihlstrasse 31,47.372904,8.536534,"[{'label': 'display', 'lat': 47.372904, 'lng':...",1270,8001,CH,Zürich,Zürich,Schweiz,"[Sihlstrasse 31 (Hotel Glockenhof), 8001 Züric...",City,Hotel Glockenhof,4f3b6e7fe4b022039bef6654


In [81]:
venues_map3 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kreis4',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map3)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe3_filtered.lat, dataframe3_filtered.lng, dataframe3_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map3)

# display map
venues_map3

In [85]:
ZRH1 = dataframe_filtered
ZRH1 = ZRH1.append (dataframe1_filtered)
ZRH1 = ZRH1.append (dataframe2_filtered)
ZRH1 = ZRH1.append (dataframe3_filtered)
ZRH1.head()
ZRH1.shape

(125, 16)

In [86]:
ZRH1.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Food from Farmers,Burger Joint,Langstrasse 211,Josefstrasse,47.368502,8.545671,"[{'label': 'display', 'lat': 47.36850193220842...",360,8005,CH,Zürich,Zürich,Schweiz,"[Langstrasse 211 (Josefstrasse), 8005 Zürich, ...",NaN,560595f7498eb0f06171956e
1,Jelmoli Food Market,Market,NaN,NaN,47.374083,8.537229,"[{'label': 'display', 'lat': 47.3740827107709,...",539,NaN,CH,NaN,NaN,Schweiz,[Schweiz],NaN,5357cc16498e0e40db63f54a
2,Hong-Kong Food Paradise,Chinese Restaurant,Seidengasse 1,NaN,47.374042,8.536562,"[{'label': 'display', 'lat': 47.37404169003569...",568,8001,CH,Zürich,Zürich,Schweiz,"[Seidengasse 1, 8001 Zürich, Schweiz]",NaN,4e87185182316be403a7f640
3,Food Station,Food Court,Bahnhofplatz 15,NaN,47.378204,8.535982,"[{'label': 'display', 'lat': 47.378204, 'lng':...",980,NaN,CH,NaN,NaN,Schweiz,"[Bahnhofplatz 15, Schweiz]",NaN,5bac7c428b98fd0039448a15
4,Coop Supermarkt City Zürich St. Annahof Food,None,Bahnhofstrasse 57,NaN,47.372902,8.538024,"[{'label': 'display', 'lat': 47.37290177, 'lng...",400,8001,CH,Zürich,Zürich,Schweiz,"[Bahnhofstrasse 57, 8001 Zürich, Schweiz]",NaN,5cc1c6a4cabcff002ce22ad4


In [88]:
venues_map4 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map4)
for lat, lng, label in zip(dataframe1_filtered.lat, dataframe1_filtered.lng, dataframe1_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map4)
for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(venues_map4)
    
for lat, lng, label in zip(dataframe3_filtered.lat, dataframe3_filtered.lng, dataframe3_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map4)
    
# display map
venues_map4